# Tool Calling & Agent

---

## 환경 설정 및 준비

`(1) Env 환경변수`

In [ ]:
from dotenv import load_dotenv
load_dotenv()

`(2) 기본 라이브러리`

In [ ]:
import os
from glob import glob

from pprint import pprint
import json

`(3) Langsmith tracing 설정`

In [ ]:
# Langsmith tracing 여부를 확인 (true: langsmith 추적 활성화, false: langsmith 추적 비활성화)
import os
print(os.getenv('LANGSMITH_TRACING'))

---

## Tool Calling 개념

- **Tool Calling**은 LLM이 외부 시스템과 상호작용하기 위한 핵심 메커니즘
- **구조화된 출력**: LLM이 정의된 스키마에 따라 함수 호출 정보를 생성
- **외부 시스템 연동**: API, 데이터베이스, 파일 시스템 등과 연결
- **자동 검증**: 스키마 기반으로 입력 파라미터 자동 검증
- **유연한 확장**: 새로운 도구를 쉽게 추가하고 제거 가능


![Tool Calling Concept](https://python.langchain.com/assets/images/tool_calling_concept-552a73031228ff9144c7d59f26dedbbf.png)


[참조] https://python.langchain.com/docs/concepts/tool_calling/

---

### 1. 기본적인 Tool 생성

#### 1.1 @tool 데코레이터 사용법

- **@tool 데코레이터**로 함수에 스키마 정보 추가

- **함수와 스키마** 간 자동 연결로 도구 생성

In [ ]:
from langchain_core.tools import tool
from typing import Literal

@tool
def add(a: int, b: int) -> int:
    """두 정수를 더합니다.
    
    Args:
        a: 첫 번째 정수
        b: 두 번째 정수
    
    Returns:
        두 수의 합
    """
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """두 정수를 곱합니다.
    
    Args:
        a: 첫 번째 정수
        b: 두 번째 정수
    
    Returns:
        두 수의 곱
    """
    return a * b

# 도구 실행 테스트
print(add.invoke({'a': 3, 'b': 5}))  # 8
print(multiply.invoke({'a': 4, 'b': 6}))  # 24

#### 1.2 복잡한 Tool 예제

In [ ]:
from datetime import datetime
from typing import Optional

@tool
def get_current_time(format_type: Literal["date", "time", "both"] = "both") -> str:
    """현재 날짜와 시간을 반환합니다.
    
    Args:
        format_type: 반환할 형식 ('date', 'time', 'both' 중 선택)
    
    Returns:
        포맷된 날짜/시간 문자열
    """
    now = datetime.now()
    
    if format_type == "date":
        return now.strftime("%Y년 %m월 %d일")
    elif format_type == "time":
        return now.strftime("%H시 %M분 %S초")
    else:
        return now.strftime("%Y년 %m월 %d일 %H시 %M분 %S초")

@tool
def calculate_age(birth_year: int) -> str:
    """태어난 년도를 입력받아 나이를 계산합니다.
    
    Args:
        birth_year: 태어난 년도 (예: 1990)
    
    Returns:
        계산된 나이
    """
    current_year = datetime.now().year
    age = current_year - birth_year
    return f"{age}세"

#### 1.3 DB 연결 Tool 예제

In [ ]:
# 벡터 저장소 로드 
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

def initialize_vector_store(embeddings = OpenAIEmbeddings(model="text-embedding-3-small"), collection_name="hybrid_search_db", persist_directory = "./local_chroma_db"):
    """
    기존 벡터 저장소를 로드하거나 새로 생성
    
    Returns:
        Chroma: 벡터 저장소 객체
    """
    try:
        
        # 기존 벡터 저장소 로드 시도
        vector_store = Chroma(
            collection_name=collection_name,
            embedding_function=embeddings,
            persist_directory=persist_directory,
        )
        
        doc_count = vector_store._collection.count()
        if doc_count > 0:
            print(f"✅ 기존 벡터 저장소 로드: {doc_count}개 문서")
            return vector_store
        else:
            print("⚠️ 빈 벡터 저장소입니다. 데이터를 추가해주세요.")
            return vector_store
            
    except Exception as e:
        print(f"❌ 벡터 저장소 로드 실패: {e}")
        return None

# 벡터 저장소 초기화
chroma_db = initialize_vector_store()

# 검색기 지정하여 테스트 
chroma_k_retriever = chroma_db.as_retriever(
    search_kwargs={"k": 2},
)

query = "리비안은 언제 사업을 시작했나요?"
retrieved_docs = chroma_k_retriever.invoke(query)

print(f"쿼리: {query}")
print("검색 결과:")
for doc in retrieved_docs:
    print(f"{doc.page_content} [출처: {doc.metadata['source']}]")
    print('-'*100)

In [ ]:
# DB 검색하는 사용자 정의 도구 생성
from langchain_core.tools import tool

@tool
def search_db(query: str):
    """리비안, 테슬라 회사에 대한 정보를 관련 데이터베이스에서 검색합니다."""
    docs = chroma_k_retriever.invoke(query)
    return "/n/n".join([doc.page_content for doc in docs])

# 도구 실행
search_db.invoke("리비안은 언제 사업을 시작했나요?")

In [ ]:
from langchain.agents.agent_toolkits import create_retriever_tool

# DB 검색하는 도구 생성
search_db = create_retriever_tool(
    chroma_k_retriever,
    name="search_db",
    description="리비안, 테슬라 회사에 대한 정보를 관련 데이터베이스에서 검색합니다.",
)

# 도구 실행
search_db.invoke("리비안은 언제 사업을 시작했나요?")

### 2. Tool을 LLM에 연결하기

In [ ]:
from langchain_openai import ChatOpenAI

# LLM 모델 초기화
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

# 도구 목록 생성
tools = [add, multiply, get_current_time, calculate_age, search_db]

# 도구를 LLM에 바인딩
llm_with_tools = llm.bind_tools(tools)

# 계산 도구 호출 테스트
response = llm_with_tools.invoke("15와 23을 더해주세요")
print(response)

In [ ]:
print(response.tool_calls)

In [ ]:
# 날짜 시간 도구 호출 테스트
response = llm_with_tools.invoke("현재 시간을 알려주세요")
print(response) 

In [ ]:
print(response.tool_calls)

In [ ]:
# 나이 계산 도구 호출 테스트
response = llm_with_tools.invoke("내가 태어난 년도는 1990년입니다. 내 나이를 알려주세요.")
print(response)

In [ ]:
print(response.tool_calls)

In [ ]:
# DB 검색 도구 호출
response = llm_with_tools.invoke("리비안은 언제 사업을 시작했나요?")
print(response)

In [ ]:
print(response.tool_calls)

### 3. Tool Calling 처리하기

#### 3.1 Tool Calls 확인하고 실행하기

In [ ]:
def execute_tool_calls(message, tools_dict):
    """Tool calls를 실행하고 도구 실행 결과인 ToolMessage 리스트를 반환합니다."""
    results = []
    
    for tool_call in message.tool_calls:

        tool_name = tool_call['name']
        
        if tool_name in tools_dict:
            tool = tools_dict[tool_name]
            tool_msg = tool.invoke(tool_call)
            results.append(tool_msg)
    
    return results

# 도구 딕셔너리 생성
tools_dict = {tool.name: tool for tool in tools}

# 테스트
response = llm_with_tools.invoke("오늘 날짜를 알려주세요")
if response.tool_calls:
    tool_messages = execute_tool_calls(response, tools_dict)
    for msg in tool_messages:
        msg.pretty_print()

In [ ]:
msg

#### 3.2 완전한 대화 흐름

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage

def chat_with_tools(user_input: str, llm_with_tools, tools_dict):
    """도구를 사용한 완전한 대화 처리"""
    messages = [HumanMessage(content=user_input)]
    
    # LLM 호출
    ai_response = llm_with_tools.invoke(messages)
    messages.append(ai_response)
    
    # Tool calls가 있으면 실행
    if ai_response.tool_calls:
        for tool_call in ai_response.tool_calls:
            tool_name = tool_call['name']            
            # 도구 실행
            if tool_name in tools_dict:
                tool_msg = tools_dict[tool_name].invoke(tool_call)
                # ToolMessage 추가
                messages.append(tool_msg)

        # 최종 응답 생성
        final_response = llm_with_tools.invoke(messages)
        return final_response.content, messages
    else:
        return ai_response.content, messages

# 테스트
result, tool_messages = chat_with_tools("15 곱하기 8은 얼마인가요?", llm_with_tools, tools_dict)
print(result)

In [ ]:
for msg in tool_messages:
    msg.pretty_print()

---

###  Tool Calling 사용 시 **고려사항**

- **모델 호환성**이 Tool Calling 성능에 직접 영향

- **명확한 도구 정의**가 모델의 이해도와 활용도 향상

- **단순한 기능**의 도구가 더 효과적으로 작동

- **과다한 도구**는 모델 성능 저하 유발

---

## Agent 개념과 실습

- **Agent**는 LLM을 의사결정 엔진으로 사용하여 복잡한 작업을 자동으로 수행하는 시스템
- Agent의 구성 요소:
    1. **LLM (추론 엔진)**: 상황을 분석하고 다음 행동을 결정
    2. **Tools (도구)**: Agent가 사용할 수 있는 기능들
    3. **Memory (메모리)**: 이전 대화나 작업 기록 저장
    4. **Prompt (프롬프트)**: Agent의 역할과 행동 지침 정의
- **LangGraph** 활용
    - **LangGraph**는 LangChain의 확장 도구로 **고급 에이전트 개발**을 지원
    - **그래프 기반 워크플로우**를 통해 복잡한 에이전트 로직을 구현할 수 있음 
    - 상태 관리와 **타입 안전성**을 통해 안정적인 에이전트 실행을 보장

In [ ]:
from langgraph.prebuilt import create_react_agent 

# 기본 React Agent 생성
tools = [add, multiply, get_current_time, calculate_age, search_db]
langgraph_agent = create_react_agent(llm, tools)

# 실행
response = langgraph_agent.invoke({
    "messages": [HumanMessage(content="25 + 17을 계산해주세요")]
})

for message in response['messages']:
    message.pretty_print()

In [ ]:
# 실행
response = langgraph_agent.invoke({
    "messages": [HumanMessage(content="리비안의 설립 과정에 대해 알려주세요.")]
})

for message in response['messages']:
    message.pretty_print()

In [ ]:
pprint(message.content)